In [2]:
import os
import json
import requests

In [32]:
url = 'https://api.ebird.org/v2/data/obs/US/recent'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 30}
response = requests.get(url, headers = header, params=params)


In [30]:
response

<Response [200]>

In [31]:
data = response.json()
data

[{'speciesCode': 'reshaw',
  'comName': 'Red-shouldered Hawk',
  'sciName': 'Buteo lineatus',
  'locId': 'L22475513',
  'locName': 'University of Florida--Field & Fork Farm and Gardens',
  'obsDt': '2024-01-13 17:58',
  'howMany': 1,
  'lat': 29.64438,
  'lng': -82.362406,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S158824868'},
 {'speciesCode': 'grycat',
  'comName': 'Gray Catbird',
  'sciName': 'Dumetella carolinensis',
  'locId': 'L21069352',
  'locName': "Mom's House",
  'obsDt': '2024-01-13 17:57',
  'howMany': 2,
  'lat': 28.5462164,
  'lng': -81.773944,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S158824774'},
 {'speciesCode': 'easowl1',
  'comName': 'Eastern Screech-Owl',
  'sciName': 'Megascops asio',
  'locId': 'L28769291',
  'locName': '6239 Schochary Road, New Tripoli, Pennsylvania, US (40.666, -75.756)',
  'obsDt': '2024-01-13 17:55',
  'howMany': 1,
  'lat': 40.6658153,
  'lng': -75.7559346,
 

In [13]:
print(header)

X-eBirdApiToken: cb88tbtnpnmp
